In [1]:
# Import necessary libraries
import pandas as pd
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

In [2]:
raw_numpy = pd.read_csv("numpy.csv")
raw_pandas = pd.read_csv("pandas.csv")
raw_csharp = pd.read_csv("csharp.csv")
raw_matlab = pd.read_csv("matlab.csv")
raw_matplotlib = pd.read_csv("matplotlib.csv")
raw_mongodb = pd.read_csv("mongodb.csv")
raw_tensorflow = pd.read_csv("tensorflow.csv")
raw_typescript = pd.read_csv("typescript.csv")

In [3]:
raw_numpy["Tag"]="numpy"
raw_pandas["Tag"]="pandas"
raw_csharp["Tag"]="csharp"
raw_matlab["Tag"]="matlab"
raw_matplotlib["Tag"]="matplotlib"
raw_mongodb["Tag"]="mongodb"
raw_tensorflow["Tag"]="tensorflow"
raw_typescript["Tag"]="typescript"

In [4]:
df = pd.concat([raw_numpy, raw_pandas, raw_csharp, raw_matlab, raw_matplotlib, raw_mongodb, raw_tensorflow, raw_typescript], axis=0)
df = df.reset_index(drop=True)
df.shape

(360718, 2)

In [5]:
def print_plot(index):
    example = df[df.index == index][['Questions', 'Tag']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)
print_plot(300000)

Sum rows of 2D array with elements of 1D array
Tag: numpy
Running automatic annotation in cvat with tensorflow results in status code 400 "No labels found for tf annotation"
Tag: tensorflow


In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['Questions'] = df['Questions'].apply(clean_text)
print_plot(10)
print_plot(300000)

c:\Users\einst\.conda\envs\tf\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


sum rows 2d array elements 1d array
Tag: numpy
running automatic annotation cvat tensorflow results status code 400 labels found tf annotation
Tag: tensorflow


In [7]:
sentences = df['Questions'].apply(lambda x: x.split()).tolist()
sentences

[['numpy',
  'savez',
  'list',
  '3d',
  'arrays',
  'different',
  'shapes',
  'fails',
  'stacking',
  'broadcasting',
  'shouldnt',
  'simply',
  'zipping',
  'separate',
  'npy',
  'files'],
 ['dataframe', 'object', 'attribute', 'close', 'pandas', 'fix', 'problem'],
 ['look', 'multiple', 'values', 'one', 'column', 'python', 'duplicate'],
 ['fastest',
  'data',
  'types',
  'use',
  'alternative',
  'c++',
  'vectors',
  'python'],
 ['rgb2hed', 'giving', 'answer', 'applied', 'full', 'part', 'matrix'],
 ['reduce', 'time', 'complexity', 'dfapply'],
 ['replace', 'double', 'loop', 'numpy'],
 ['load', 'txt', 'numpy', '2d', 'array', 'dictionaries', 'objects'],
 ['using', 'to_pickle', 'store', 'large', 'dataframes'],
 ['iam',
  'studyng',
  'data',
  'science',
  'dont',
  'know',
  'make',
  'labels',
  'x',
  'dont',
  'stay',
  'together',
  'duplicate'],
 ['sum', 'rows', '2d', 'array', 'elements', '1d', 'array'],
 ['matplotlib', 'get', 'precise', 'point', 'intersection'],
 ['find', 'p

In [8]:
# Split your dataset into training and testing sets
train_sentences, test_sentences = train_test_split(sentences, test_size=0.2, random_state=42)

In [9]:
w2v_model = Word2Vec(sentences=train_sentences, window=5, vector_size=300, min_count=1, workers=4)

In [11]:
w2v_model.wv.most_similar(positive=["mvvm"])

[('xaml', 0.9265304207801819),
 ('viewmodel', 0.9149225354194641),
 ('usercontrol', 0.9061193466186523),
 ('wpf', 0.8958867788314819),
 ('xamarin', 0.8921669125556946),
 ('uwp', 0.8916560411453247),
 ('controls', 0.8884421586990356),
 ('selecteditem', 0.881190299987793),
 ('winforms', 0.8807506561279297),
 ('webview2', 0.8798460960388184)]

In [16]:
# Train logistic regression model
# Convert sentences into word embeddings using word2vec model
train_X = [w2v_model.wv[key] for sentence in train_sentences for key in sentence if key in w2v_model.wv.key_to_index]
train_y = df.loc[df['Questions'].isin(train_sentences), 'Tag']
test_X = [w2v_model.wv[key] for sentence in test_sentences for key in sentence if key in w2v_model.wv.key_to_index]
test_y = df.loc[df['Questions'].isin(test_sentences), 'Tag']

In [ ]:
# Create a logistic regression model
clf = LogisticRegression(max_iter=1000)

In [ ]:
# Fit the logistic regression model on training data
clf.fit(train_X, train_y)

In [ ]:
# Make predictions on test data
y_pred = clf.predict(test_X)